<a href="https://colab.research.google.com/github/nithyavision/ai-solutins/blob/main/bad_shrimp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import torch


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.8/882.8 kB 23.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np


model = YOLO('/content/drive/MyDrive/fishery_data/best (1).pt')


def detect_objects(frame):
    results = model(frame)
    detected_objects = results[0].boxes
    confidences = detected_objects.conf.cpu().numpy()
    bboxes = detected_objects.xyxy.cpu().numpy()


    valid_detections = bboxes[confidences >= 0.5]

    return valid_detections


def draw_bounding_boxes(frame, detections):
    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
    return frame

def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        detections = detect_objects(frame)


        if detections.size > 0:
            frame = draw_bounding_boxes(frame, detections)


        out.write(frame)


    cap.release()
    out.release()
    print("Video processing complete. Output saved at:", output_video_path)


input_video_path = '/content/drive/MyDrive/fishery_data/short_shrimp_cleaning.mp4'
output_video_path = '/content/short_shrimp_output.mp4'
process_video(input_video_path, output_video_path)


Streaming output truncated to the last 5000 lines.
Speed: 4.7ms preprocess, 9.0ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 26 badshrimps, 9.7ms
Speed: 4.1ms preprocess, 9.7ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 25 badshrimps, 8.7ms
Speed: 5.8ms preprocess, 8.7ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 25 badshrimps, 7.1ms
Speed: 3.5ms preprocess, 7.1ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 25 badshrimps, 12.9ms
Speed: 3.8ms preprocess, 12.9ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 25 badshrimps, 15.0ms
Speed: 9.2ms preprocess, 15.0ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 25 badshrimps, 8.2ms
Speed: 3.7ms preprocess, 8.2ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 25 badshrimps, 7.8ms
Speed: 4.1ms preprocess, 7.8ms inference, 

Detction on image based on model

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO('/content/drive/MyDrive/best.pt')
# Step 2: Perform object detection
def detect_objects(image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Detect objects

    # Check if results is a list and contains any detections
    if isinstance(results, list) and len(results) > 0:
        detected_objects = results[0].boxes.xyxy.cpu().numpy()  # Access bounding boxes from the first element of the list
    else:
        detected_objects = np.array([])  # Return an empty array if no objects are detected

    return image, detected_objects

# Step 3: Crop detected objects
def crop_objects(image, detections):
    crops = []
    for det in detections:
        x1, y1, x2, y2 = det
        crop = image[int(y1):int(y2), int(x1):int(x2)]
        crops.append(crop)
    return crops

# Step 4: Detect white cloudy marks (using color thresholding)
def detect_white_cloudy_marks(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])

    # Apply the threshold for white colors
    mask = cv2.inRange(hsv_image, lower_white, upper_white)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Optional: Use Canny edge detection to refine the marks
    edges = cv2.Canny(result, 100, 200)

    return result, edges

# Step 5: Draw bounding boxes and save the final image
def draw_bounding_boxes(image, detections):
    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])  # Get bounding box coordinates
        label = "Detected Object"  # You can customize the label or extract it from the results

        # Draw rectangle (bounding box) around detected object
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Add label text above the bounding box
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return image

# Main execution
image_path = '/content/shrimp1.png'
image, detections = detect_objects(image_path)

# Step 6: Check for detections and process them
if detections.size > 0:  # Check if any objects were detected
    # Step 7: Draw bounding boxes and save the output image
    image_with_boxes = draw_bounding_boxes(image, detections)

    # Save the result as an output image
    output_image_path = 'output_image.png'
    cv2.imwrite(output_image_path, image_with_boxes)
    print(f"Output image saved at: {output_image_path}")

    # Display the image with bounding boxes
    #cv2.imshow("Detected Objects", image_with_boxes)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print("No objects detected in the image.")



0: 416x640 6 shrimps, 9.6ms
Speed: 2.0ms preprocess, 9.6ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)
Output image saved at: output_image.png


Detection on image based on cloudy


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np # Added import for numpy

# Step 2: Perform object detection
def detect_objects(image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Detect objects

    # Check if results is a list and contains any detections
    if isinstance(results, list) and len(results) > 0:
        detected_objects = results[0].boxes.xyxy.cpu().numpy()  # Access bounding boxes from the first element of the list
    else:
        detected_objects = np.array([]) # Return an empty array if no objects are detected

    return image, detected_objects

# Step 3: Crop detected objects
def crop_objects(image, detections):
    crops = []
    for det in detections:
        x1, y1, x2, y2 = det
        crop = image[int(y1):int(y2), int(x1):int(x2)]
        crops.append(crop)
    return crops

# Step 4: Detect white cloudy marks (using color thresholding)
def detect_white_cloudy_marks(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])

    # Apply the threshold for white colors
    mask = cv2.inRange(hsv_image, lower_white, upper_white)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Optional: Use Canny edge detection to refine the marks
    edges = cv2.Canny(result, 100, 200)

    return result, edges

# Main execution
image_path = '/content/shrimp1.png'
image, detections = detect_objects(image_path)

# Step 5: Crop objects and check for white cloudy marks
if detections.size > 0: # Check if any objects were detected
    for crop in crop_objects(image, detections):
        result, edges = detect_white_cloudy_marks(crop)

        # Display the result
        #cv2.imshow("Cloudy Marks", result)

        #cv2.imshow("Edges", edges)
        cv2.waitKey(0)

    cv2.destroyAllWindows()
else:
    print("No objects detected in the image.")

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np  # Added import for numpy

# Load the YOLO model
model = YOLO('/content/drive/MyDrive/best.pt')

# Step 2: Perform object detection
def detect_objects(image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Detect objects

    # Check if results contain any detections
    if isinstance(results, list) and len(results) > 0:
        detected_objects = results[0].boxes.xyxy.cpu().numpy()  # Access bounding boxes from the first element of the list
    else:
        detected_objects = np.array([])  # Return an empty array if no objects are detected

    return image, detected_objects

# Step 3: Crop detected objects
def crop_objects(image, detections):
    crops = []
    for i, det in enumerate(detections):
        x1, y1, x2, y2 = det[:4]  # Extract bounding box coordinates
        crop = image[int(y1):int(y2), int(x1):int(x2)]
        crops.append((crop, i))  # Save the crop along with the index for naming
    return crops

# Step 4: Detect white cloudy marks (using color thresholding)
def detect_white_cloudy_marks(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])

    # Apply the threshold for white colors
    mask = cv2.inRange(hsv_image, lower_white, upper_white)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Optional: Use Canny edge detection to refine the marks
    edges = cv2.Canny(result, 100, 200)

    return result, edges

# Main execution
image_path = '/content/shrimp1.png'
image, detections = detect_objects(image_path)

# Step 5: Crop objects and check for white cloudy marks
if detections.size > 0:  # Check if any objects were detected
    for crop, idx in crop_objects(image, detections):
        result, edges = detect_white_cloudy_marks(crop)

        # Save the results
        output_crop_path = f'crop_{idx}.png'
        output_result_path = f'result_{idx}.png'
        output_edges_path = f'edges_{idx}.png'

        cv2.imwrite(output_crop_path, crop)  # Save cropped object
        cv2.imwrite(output_result_path, result)  # Save result with white cloudy marks
        cv2.imwrite(output_edges_path, edges)  # Save edges image

    print("Images saved successfully.")
else:
    print("No objects detected in the image.")


WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 6.6ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 384x640 (no detections), 8.7ms
Speed: 1.8ms preprocess, 7.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
No objects detected in the image.


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np


model = YOLO('/content/drive/MyDrive/best.pt')

def detect_objects(image_path):
    image = cv2.imread(image_path)
    results = model(image)


    if isinstance(results, list) and len(results) > 0:
        detected_objects = results[0].boxes.xyxy.cpu().numpy()
    else:
        detected_objects = np.array([])

    return image, detected_objects


def draw_bounding_boxes(image, detections):
    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image

def detect_white_cloudy_marks(image, crop, x1, y1, x2, y2):
    hsv_image = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])

    mask = cv2.inRange(hsv_image, lower_white, upper_white)
    result = cv2.bitwise_and(crop, crop, mask=mask)


    image[y1:y2, x1:x2] = cv2.addWeighted(image[y1:y2, x1:x2], 0.7, result, 0.3, 0)

    return image


image_path = '/content/shrimp1.png'
image, detections = detect_objects(image_path)


if detections.size > 0:
    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])
        crop = image[y1:y2, x1:x2]
        image = detect_white_cloudy_marks(image, crop, x1, y1, x2, y2)


    image = draw_bounding_boxes(image, detections)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No objects detected in the image.")



0: 416x640 6 shrimps, 158.0ms
Speed: 4.8ms preprocess, 158.0ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load the YOLO model
model = YOLO('/content/drive/MyDrive/best.pt')

# Step 1: Perform object detection
def detect_objects(frame):
    results = model(frame)  # Detect objects
    # Check if results contain any detections
    if isinstance(results, list) and len(results) > 0:
        detected_objects = results[0].boxes.xyxy.cpu().numpy()  # Access bounding boxes from the first element of the list
    else:
        detected_objects = np.array([])  # Return an empty array if no objects are detected
    return detected_objects

# Step 2: Draw bounding boxes on the frame
def draw_bounding_boxes(frame, detections):
    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])  # Extract bounding box coordinates
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw the rectangle
    return frame

# Step 3: Detect white cloudy marks
def detect_white_cloudy_marks(frame, crop, x1, y1, x2, y2):
    hsv_image = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])

    # Apply the threshold for white colors
    mask = cv2.inRange(hsv_image, lower_white, upper_white)
    result = cv2.bitwise_and(crop, crop, mask=mask)

    # Blend the result back onto the original frame at the detected bounding box
    frame[y1:y2, x1:x2] = cv2.addWeighted(frame[y1:y2, x1:x2], 0.7, result, 0.3, 0)

    return frame

# Step 4: Process video frame by frame and save the output
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)  # Capture the video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for the output video
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get the original video's FPS
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Get frame width
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Get frame height

    # Set up video writer to save the output video
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop the loop when the video ends

        # Detect objects in the current frame
        detections = detect_objects(frame)

        # If objects are detected, process each object
        if detections.size > 0:
            for det in detections:
                x1, y1, x2, y2 = map(int, det[:4])  # Get bounding box coordinates
                crop = frame[y1:y2, x1:x2]  # Crop the detected object
                frame = detect_white_cloudy_marks(frame, crop, x1, y1, x2, y2)  # Detect and blend cloudy marks

            # Draw bounding boxes on the frame
            frame = draw_bounding_boxes(frame, detections)

        # Write the processed frame to the output video
        out.write(frame)

    # Release the video objects
    cap.release()
    out.release()
    print("Video processing complete. Output saved at:", output_video_path)

# Main execution
input_video_path = '/content/drive/MyDrive/scleaning.ts'  # Path to your input video
output_video_path = '/content/shrimp_output_video.mp4'  # Path to save the output video

process_video(input_video_path, output_video_path)


Streaming output truncated to the last 5000 lines.
Speed: 3.1ms preprocess, 12.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.3ms
Speed: 3.1ms preprocess, 11.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.5ms
Speed: 3.1ms preprocess, 11.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.0ms
Speed: 3.3ms preprocess, 11.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 18.3ms
Speed: 3.1ms preprocess, 18.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.3ms
Speed: 3.4ms preprocess, 9.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.7ms
Speed: 3.2ms preprocess, 11.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.4ms
Speed: 7.5ms preproc

In [ ]:


# Step 2: Perform object detection
def detect_objects(image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Detect objects
    detected_objects = results.xyxy[0].numpy()  # Get bounding box coordinates
    return image, detected_objects

# Step 3: Crop detected objects
def crop_objects(image, detections):
    crops = []
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        crop = image[int(y1):int(y2), int(x1):int(x2)]
        crops.append(crop)
    return crops

# Step 4: Detect white cloudy marks (using color thresholding)
def detect_white_cloudy_marks(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])

    # Apply the threshold for white colors
    mask = cv2.inRange(hsv_image, lower_white, upper_white)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Optional: Use Canny edge detection to refine the marks
    edges = cv2.Canny(result, 100, 200)

    return result, edges

# Main execution
image_path = '/content/shrim.jpg'
image, detections = detect_objects(image_path)

# Step 5: Crop objects and check for white cloudy marks
for crop in crop_objects(image, detections):
    result, edges = detect_white_cloudy_marks(crop)

    # Display the result
    cv2.imshow("Cloudy Marks", result)
    cv2.imshow("Edges", edges)
    cv2.waitKey(0)

cv2.destroyAllWindows()



0: 640x544 (no detections), 209.9ms
Speed: 6.5ms preprocess, 209.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 544)


AttributeError: 'list' object has no attribute 'xyxy'

In [ ]:


# Function to run detection on an image
def detect_on_image(image_path, save_path=None):
    results = model(image_path)  # Perform detection
    results.show()  # Show the image with detections

    if save_path:
        results.save(save_path)  # Save the image with detections

# Function to run detection on a video
def detect_on_video(video_path, save_path=None):
    results = model(video_path, show=True)  # Perform detection and display live

    if save_path:
        results.save(save_path)  # Save the video with detections

# Function to run detection using webcam
def detect_on_webcam():
    model(source=0, show=True)  # 0 indicates using the default webcam

In [ ]:
detect_on_image('shrim.jpg', save_path='image_output')



image 1/1 /content/shrim.jpg: 640x544 (no detections), 252.3ms
Speed: 3.9ms preprocess, 252.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 544)


AttributeError: 'list' object has no attribute 'show'

In [ ]:


# Example: Run detection on a video
detect_on_video('path/to/video.mp4', save_path='path/to/save_video_output')

# Example: Run detection on a webcam
# detect_on_webcam()


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/custom_model.pt'

In [ ]:
from ultralytics import YOLO
import cv2

# Load the custom YOLOv8 model
model = YOLO('path/to/your/custom_model.pt')

# Function to run detection on an image
def detect_on_image(image_path, save_path=None):
    results = model(image_path)  # Perform detection
    results.show()  # Show the image with detections

    if save_path:
        results.save(save_path)  # Save the image with detections

# Function to run detection on a video
def detect_on_video(video_path, save_path=None):
    results = model(video_path, show=True)  # Perform detection and display live

    if save_path:
        results.save(save_path)  # Save the video with detections

# Function to run detection using webcam
def detect_on_webcam():
    model(source=0, show=True)  # 0 indicates using the default webcam

# Example: Run detection on an image
detect_on_image('path/to/image.jpg', save_path='path/to/save_image_output')

# Example: Run detection on a video
detect_on_video('path/to/video.mp4', save_path='path/to/save_video_output')

# Example: Run detection on a webcam
# detect_on_webcam()


In [ ]:
import cv2
import numpy as np
import torch

# Step 1: Load a YOLOv5 model (or use another detection model)
model = torch.hub.load('/content/drive/MyDrive/best.pt')  # Pre-trained YOLOv5 model

# Step 2: Perform object detection/content/drive/MyDrive/best.pt
def detect_objects(image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Detect objects
    detected_objects = results.xyxy[0].numpy()  # Get bounding box coordinates
    return image, detected_objects

# Step 3: Crop detected objects
def crop_objects(image, detections):
    crops = []
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        crop = image[int(y1):int(y2), int(x1):int(x2)]
        crops.append(crop)
    return crops

# Step 4: Detect white cloudy marks (using color thresholding)
def detect_white_cloudy_marks(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])

    # Apply the threshold for white colors
    mask = cv2.inRange(hsv_image, lower_white, upper_white)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Optional: Use Canny edge detection to refine the marks
    edges = cv2.Canny(result, 100, 200)

    return result, edges

# Main execution
image_path = 'path_to_image.jpg'
image, detections = detect_objects(image_path)

# Step 5: Crop objects and check for white cloudy marks
for crop in crop_objects(image, detections):
    result, edges = detect_white_cloudy_marks(crop)

    # Display the result
    cv2.imshow("Cloudy Marks", result)
    cv2.imshow("Edges", edges)
    cv2.waitKey(0)

cv2.destroyAllWindows()


TypeError: load() missing 1 required positional argument: 'model'

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load the YOLO model
model = YOLO('/content/drive/MyDrive/best.pt')

# Step 1: Perform object detection
def detect_objects(frame):
    results = model(frame)  # Detect objects
    # Check if results contain any detections
    if isinstance(results, list) and len(results) > 0:
        detected_objects = results[0].boxes.xyxy.cpu().numpy()  # Access bounding boxes from the first element of the list
    else:
        detected_objects = np.array([])  # Return an empty array if no objects are detected
    return detected_objects

# Step 2: Draw bounding boxes on the frame
def draw_bounding_boxes(frame, detections):
    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])  # Extract bounding box coordinates
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw the rectangle
    return frame

#

# Step 4: Process video frame by frame and save the output
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)  # Capture the video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for the output video
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get the original video's FPS
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Get frame width
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Get frame height

    # Set up video writer to save the output video
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop the loop when the video ends

        # Detect objects in the current frame
        detections = detect_objects(frame)

        # If objects are detected, process each object
        if detections.size > 0:
            for det in detections:
                x1, y1, x2, y2 = map(int, det[:4])  # Get bounding box coordinates
                crop = frame[y1:y2, x1:x2]  # Crop the detected object
                frame = detect_white_cloudy_marks(frame, crop, x1, y1, x2, y2)  # Detect and blend cloudy marks

            # Draw bounding boxes on the frame
            frame = draw_bounding_boxes(frame, detections)

        # Write the processed frame to the output video
        out.write(frame)

    # Release the video objects
    cap.release()
    out.release()
    print("Video processing complete. Output saved at:", output_video_path)

# Main execution
input_video_path = '/content/drive/MyDrive/scleaning.ts'  # Path to your input video
output_video_path = '/content/shrimp_output_video.mp4'  # Path to save the output video

process_video(input_video_path, output_video_path)
